In [1]:
import pandas as pd
import re
import ast

# Attacks Conversion

In [3]:
unit_costs = {
    "Protoss Probe": {"minerals":50,"gas":0},
    "Protoss Photon Cannon": {"minerals":150,"gas":0},
    "Protoss Zealot": {"minerals":100,"gas":0},
    "Protoss Dragoon": {"minerals":125,"gas":50},
    "Protoss High Templar": {"minerals":50,"gas":150},
    "Protoss Dark Templar": {"minerals":125,"gas":100},
    "Protoss Archon": {"minerals":100,"gas":300},
    "Protoss Dark Archon": {"minerals":250,"gas":200},
    "Protoss Scout": {"minerals":275,"gas":125},
    "Protoss Corsair": {"minerals":150,"gas":100},
    "Protoss Carrier": {"minerals":350,"gas":250},
    "Protoss Interceptor": {"minerals":25,"gas":0},
    "Protoss Arbiter": {"minerals":100,"gas":350},
    "Protoss Reaver": {"minerals":200,"gas":100},
    "Protoss Observer": {"minerals":25,"gas":75},
    "Protoss Shuttle": {"minerals":200,"gas":0},
    "Terran SCV": {"minerals":50,"gas":0},
    "Terran Missile Turret": {"minerals":75,"gas":0},
    "Terran Marine": {"minerals":50,"gas":0},
    "Terran Firebat": {"minerals":50,"gas":25},
    "Terran Ghost": {"minerals":25,"gas":75},
    "Terran Medic": {"minerals":50,"gas":25},
    "Terran Vulture": {"minerals":75,"gas":0},
    "Terran Spider Mine": {"minerals":100,"gas":100},
    "Terran Tank": {"minerals":150,"gas":100},
    "Terran Sieged Tank": {"minerals":150,"gas":150},
    "Terran Goliath": {"minerals":100,"gas":50},
    "Terran Wraith": {"minerals":150,"gas":100},
    "Terran Battle Cruiser": {"minerals":400,"gas":300},
    "Terran Science Vessel": {"minerals":100,"gas":225},
    "Terran Valkyrie": {"minerals":250,"gas":125},
    "Terran Dropship": {"minerals":100,"gas":100},
    "Zerg Drone": {"minerals":50,"gas":0},
    "Zerg Overlord": {"minerals":100,"gas":0},
    "Zerg Sunken Colony": {"minerals":175,"gas":0},
    "Zerg Spore Colony": {"minerals":175,"gas":0},
    "Zerg Zergling": {"minerals":25,"gas":0},
    "Zerg Hydralisk": {"minerals":75,"gas":25},
    "Zerg Lurker": {"minerals":125,"gas":125},
    "Zerg Queen": {"minerals":100,"gas":100},
    "Zerg Broodling": {"minerals":100,"gas":100},
    "Zerg Infested Terran": {"minerals":100,"gas":50},
    "Zerg Scourge": {"minerals":12,"gas":38},
    "Zerg Mutalisk": {"minerals":100,"gas":100},
    "Zerg Guardian": {"minerals":150,"gas":200},
    "Zerg Devourer": {"minerals":250,"gas":150},
    "Zerg Defiler": {"minerals":50,"gas":150},
    "Zerg Ultralisk": {"minerals":200,"gas":200},

}

In [22]:
def calculate_resource_loss(involved_units, remaining_units):
  player_1_minerals_lost = 0
  player_1_gas_lost = 0
  player_2_minerals_lost = 0
  player_2_gas_lost = 0

  for player, unit_counts in involved_units.items():
    for unit_type, lost_count in unit_counts.items():
      remaining_count = remaining_units.get(player, {}).get(unit_type, 0)
      lost_count = max(0, lost_count - remaining_count)

      unit_cost = unit_costs.get(unit_type, {})
      minerals_lost = lost_count * unit_cost.get("minerals", 0)
      gas_lost = lost_count * unit_cost.get("gas", 0)

      if player == 0:
        player_1_minerals_lost += minerals_lost
        player_1_gas_lost += gas_lost
      else:
        player_2_minerals_lost += minerals_lost
        player_2_gas_lost += gas_lost

  return {
      "Player1_LostInAttackMinerals": player_1_minerals_lost,
      "Player1_LostInAttackGas": player_1_gas_lost,
      "Player2_LostInAttackMinerals": player_2_minerals_lost,
      "Player2_LostInAttackGas": player_2_gas_lost,
  }

# Example usage
involved_units = {0: {'Protoss Zealot': 2, 'Protoss Dragoon': 1}, 1: {'Protoss Probe': 1, 'Protoss Zealot': 5, 'Protoss Dragoon': 2, 'Protoss Photon Cannon': 1}}

remaining_units = {0: {'Protoss Zealot': 2, 'Protoss Dragoon': 1}, 1: {'Protoss Probe': 1, 'Protoss Zealot': 5, 'Protoss Dragoon': 2, 'Protoss Photon Cannon': 1}}

resource_loss = calculate_resource_loss(involved_units, remaining_units)

print(resource_loss)


{'Player1_LostInAttackMinerals': 0, 'Player1_LostInAttackGas': 0, 'Player2_LostInAttackMinerals': 0, 'Player2_LostInAttackGas': 0}


In [35]:
import os
import pandas as pd


def parse_rgd_file(rgd_content, replay_id):
    attacked = []
    for line in rgd_content:
        action_info = line.strip().split(",")
        # print(action_info)

        if (len(action_info)>2):

            if (
                action_info[2].strip() == "IsAttacked"
            ):
                # print(action_info)
                content = re.sub(r'\b([A-Za-z_]\w*)\b', r'"\1"', line)
                content = content.replace('"winner":', '')
                content = content.replace('" "', ' ')
                content = content.replace(",),",',("Unknown"),')
                content = content.replace('-1.#"IND"','0')
                content = "["+content+"]"
                # print(content)
                list_from_string = eval(content)
                # print(type(list_from_string))
                
                winner = 0
                if sum(list_from_string[5][0].values()) - sum(list_from_string[7][0].values()) > sum(list_from_string[5][1].values()) - sum(list_from_string[7][1].values()):
                    winner = 1
                actual = list_from_string[-1]
                if(actual > 1):
                    actual = winner
                # print("0<",list_from_string)
                # print("1<",list_from_string[5])
                # print("1<",list_from_string[5][0])
                # print("2<",list_from_string[7])
                endFrame = list_from_string[0] if str(list_from_string[-1]).startswith("w") else list_from_string[-2]
                if type(endFrame) == dict:
                    endFrame = list_from_string[-1]
                # print(list_from_string[0], list_from_string[-2], list_from_string[-1])
                attacked.append(
                    {
                        "Frame" : endFrame,
                        "DefenderID" : list_from_string[1] + 1,
                        "ActionType": list_from_string[2],
                        "Player1_BeforeAttackUnits" : sum(list_from_string[5][0].values()),
                        "Player2_BeforeAttackUnits" : sum(list_from_string[5][1].values()),
                        "Player1_AfterAttackUnits" : sum(list_from_string[7][0].values()),
                        "Player2_AfterAttackUnits" : sum(list_from_string[7][1].values()),
                        "Player1_LostInAttackUnits" : sum(list_from_string[5][0].values()) - sum(list_from_string[7][0].values()),
                        "Player2_LostInAttackUnits" : sum(list_from_string[5][1].values()) - sum(list_from_string[7][1].values()),
                        **calculate_resource_loss(list_from_string[5],list_from_string[7]),
                        "AttackWinner" :actual + 1,
                    }
                )
                # print(attacked)

    return attacked


# Get a list of all .rgd files in the directory
rgd_files = [file for file in os.listdir() if file.endswith(".rgd")]

count = 0
# Process each .rgd file
for rgd_file_name in rgd_files:
    print(f"Processing file: {rgd_file_name}")

    with open(rgd_file_name, "r", encoding="latin-1") as rgd_file:
        rgd_content = rgd_file.readlines()

    # Update ReplayID with the current file name (without extension)
    replay_id = os.path.splitext(rgd_file_name)[0]
    replay_name = replay_id.split(".")[0]
    attacked = parse_rgd_file(rgd_content, replay_name)

    # Sample code for creating DataFrames
    attacked_df = pd.DataFrame(attacked)
    if attacked_df.empty:
        attacked_df = pd.DataFrame(
            columns=[
                "Frame",
                "DefenderID",
                "ActionType",
                "Player1_BeforeAttackUnits",
                "Player2_BeforeAttackUnits",
                "Player1_AfterAttackUnits",
                "Player2_AfterAttackUnits",
                "Player1_LostInAttackUnits",
                "Player2_LostInAttackUnits",
                "Player1_LostInAttackMinerals",
                "Player2_LostInAttackMinerals",
                "Player2_LostInAttackGas",
                "Player1_LostInAttackGas",
                "AttackWinner",
            ]
        )
    # Save DataFrames to CSV files
    attacked_df.to_csv(f"{replay_name}_attacks.csv", index=False)


    print(f"File {rgd_file_name} processed.\n")
    count+= 1
    # print(attacked_df)
    # break
print("Total Files Processed: ", count)

Processing file: GG10.rep.rgd
File GG10.rep.rgd processed.

Processing file: GG10042.rep.rgd
File GG10042.rep.rgd processed.

Processing file: GG10043.rep.rgd
File GG10043.rep.rgd processed.

Processing file: GG10056.rep.rgd
File GG10056.rep.rgd processed.

Processing file: GG1010.rep.rgd
File GG1010.rep.rgd processed.

Processing file: GG10208.rep.rgd
File GG10208.rep.rgd processed.

Processing file: GG10209.rep.rgd
File GG10209.rep.rgd processed.

Processing file: GG1059.rep.rgd
File GG1059.rep.rgd processed.

Processing file: GG10646.rep.rgd
File GG10646.rep.rgd processed.

Processing file: GG1066.rep.rgd
File GG1066.rep.rgd processed.

Processing file: GG10951.rep.rgd
File GG10951.rep.rgd processed.

Processing file: GG11.rep.rgd
File GG11.rep.rgd processed.

Processing file: GG11048.rep.rgd
File GG11048.rep.rgd processed.

Processing file: GG11049.rep.rgd
File GG11049.rep.rgd processed.

Processing file: GG11050.rep.rgd
File GG11050.rep.rgd processed.

Processing file: GG11091.rep

# Morphs Conversion

In [5]:
import os
import pandas as pd


def parse_rgd_file(rgd_content, replay_id):
    morphed = []
    for line in rgd_content:
        action_info = line.strip().split(",")
        # print(action_info)

        if (len(action_info)>2):

            if (
                action_info[2].strip() == "Morph"
            ):
                # print(action_info)
                morphed.append(
                    {
                        "Frame" : action_info[0],
                        "PlayerID" : int(action_info[1]) + 1,
                        "ActionType": action_info[2],
                        "UnitID": action_info[3],
                        "UnitType" : action_info[4],
                    }
                )
                # print(morphed)

    return morphed


# Get a list of all .rgd files in the directory
rgd_files = [file for file in os.listdir() if file.endswith(".rgd")]

count = 0
# Process each .rgd file
for rgd_file_name in rgd_files:
    print(f"Processing file: {rgd_file_name}")

    with open(rgd_file_name, "r", encoding="latin-1") as rgd_file:
        rgd_content = rgd_file.readlines()

    # Update ReplayID with the current file name (without extension)
    replay_id = os.path.splitext(rgd_file_name)[0]
    replay_name = replay_id.split(".")[0]
    morphed = parse_rgd_file(rgd_content, replay_name)

    # Sample code for creating DataFrames
    morphed_df = pd.DataFrame(morphed)

    if morphed_df.empty:
        morphed_df = pd.DataFrame(
            columns=[
                "Frame",
                "PlayerID",
                "ActionType",
                "UnitID",
                "UnitType",
            ]
        )
    # Save DataFrames to CSV files
    morphed_df.to_csv(f"{replay_name}_morphs.csv", index=False)


    print(f"File {rgd_file_name} processed.\n")
    count+= 1
    # print(morphed_df)

print("Total Files Processed: ", count)

Processing file: GG10.rep.rgd
File GG10.rep.rgd processed.

Processing file: GG10042.rep.rgd
File GG10042.rep.rgd processed.

Processing file: GG10043.rep.rgd
File GG10043.rep.rgd processed.

Processing file: GG10056.rep.rgd
File GG10056.rep.rgd processed.

Processing file: GG1010.rep.rgd
File GG1010.rep.rgd processed.

Processing file: GG10208.rep.rgd
File GG10208.rep.rgd processed.

Processing file: GG10209.rep.rgd
File GG10209.rep.rgd processed.

Processing file: GG1059.rep.rgd
File GG1059.rep.rgd processed.

Processing file: GG10646.rep.rgd
File GG10646.rep.rgd processed.

Processing file: GG1066.rep.rgd
File GG1066.rep.rgd processed.

Processing file: GG10951.rep.rgd
File GG10951.rep.rgd processed.

Processing file: GG11.rep.rgd
File GG11.rep.rgd processed.

Processing file: GG11048.rep.rgd
File GG11048.rep.rgd processed.

Processing file: GG11049.rep.rgd
File GG11049.rep.rgd processed.

Processing file: GG11050.rep.rgd
File GG11050.rep.rgd processed.

Processing file: GG11091.rep

# Upgrades Conversion

In [47]:
import pandas as pd

# Process each .rgd file
for rgd_file_name in rgd_files:
    print(f"Processing file: {rgd_file_name}")

    with open(rgd_file_name, "r", encoding="latin-1") as rgd_file:
        rgd_content = rgd_file.readlines()

    upgrade_actions = []
    # Process each line in the file
    for line in rgd_content:
        # Split the line by tab to extract columns
        columns = line.strip().split(",")

        # Check if the ActionType is "StartUpgrade" or "FinishUpgrade"
        if len(columns) > 2:
            columns = line.strip().split(",")
            # Check if the ActionType is "StartUpgrade"
            if columns[2] == "StartUpgrade":
                # Extract relevant information for StartUpgrade action
                frame = int(columns[0])
                player_id = int(columns[1])
                upgrade_type = columns[3]
                upgrade_level = int(columns[4])
                start_frame = int(columns[0])
                # Append StartUpgrade data to the list
                upgrade_actions.append({
                    "PlayerID": player_id + 1,
                    "ActionType": "Upgrade",
                    "UpgradeType": upgrade_type,
                    "UpgradeLevel": upgrade_level,
                    "StartFrame": start_frame,
                    "EndFrame": -1
                })
            # Check if the ActionType is "FinishUpgrade"
            elif columns[2] == "FinishUpgrade":
                # Find the corresponding StartUpgrade action
                for i in range(len(upgrade_actions)):
                    if upgrade_actions[i]["PlayerID"] == int(columns[1]) and upgrade_actions[i]["UpgradeType"] == columns[3] and upgrade_actions[i]["UpgradeLevel"] == int(columns[4]):
                        # Update the corresponding StartUpgrade action with FinishUpgrade data
                        upgrade_actions[i]["EndFrame"] = int(columns[0])


    # Convert the data to a DataFrame
    df = pd.DataFrame(upgrade_actions)

    # Print the resulting DataFrame
    print(df)
    # break


Processing file: GG10.rep.rgd
Processing file: GG10042.rep.rgd
Processing file: GG10043.rep.rgd
Processing file: GG10056.rep.rgd
Processing file: GG1010.rep.rgd
Processing file: GG10208.rep.rgd
Processing file: GG10209.rep.rgd
Processing file: GG1059.rep.rgd
Processing file: GG10646.rep.rgd
Processing file: GG1066.rep.rgd
Processing file: GG10951.rep.rgd
Processing file: GG11.rep.rgd
Processing file: GG11048.rep.rgd
Processing file: GG11049.rep.rgd
Processing file: GG11050.rep.rgd
Processing file: GG11091.rep.rgd
Processing file: GG114.rep.rgd
Processing file: GG1140.rep.rgd
Processing file: GG11478.rep.rgd
Processing file: GG11768.rep.rgd
Processing file: GG11823.rep.rgd
Processing file: GG11898.rep.rgd
Processing file: GG11981.rep.rgd
Processing file: GG12039.rep.rgd
Processing file: GG12068.rep.rgd
Processing file: GG12197.rep.rgd
Processing file: GG1226.rep.rgd
Processing file: GG12272.rep.rgd
Processing file: GG12299.rep.rgd
Processing file: GG12388.rep.rgd
Processing file: GG1264

In [52]:
import os
import pandas as pd


def parse_rgd_file(rgd_content, replay_id):
    upgrade_actions = []
    for line in rgd_content:
        action_info = line.strip().split(",")
        # print(action_info)

        if (len(action_info)>2):
            columns = action_info
            # Check if the ActionType is "StartUpgrade"
            if columns[2] == "StartUpgrade":
                # Extract relevant information for StartUpgrade action
                frame = int(columns[0])
                player_id = int(columns[1])
                upgrade_type = columns[3]
                upgrade_level = int(columns[4])
                start_frame = int(columns[0])
                # Append StartUpgrade data to the list
                upgrade_actions.append({
                    "PlayerID": player_id + 1,
                    "ActionType": "Upgrade",
                    "UpgradeType": upgrade_type,
                    "UpgradeLevel": upgrade_level,
                    "StartFrame": start_frame,
                    "EndFrame": -1
                })
            # Check if the ActionType is "FinishUpgrade"
            elif columns[2] == "FinishUpgrade":
                # Find the corresponding StartUpgrade action
                # print(upgrade_actions)
                for i in range(len(upgrade_actions)):
                    if upgrade_actions[i]["PlayerID"] == int(columns[1]) + 1 and upgrade_actions[i]["UpgradeType"] == columns[3] and upgrade_actions[i]["UpgradeLevel"] == int(columns[4]):
                        # Update the corresponding StartUpgrade action with FinishUpgrade data
                        upgrade_actions[i]["EndFrame"] = int(columns[0])

    return upgrade_actions


# Get a list of all .rgd files in the directory
rgd_files = [file for file in os.listdir() if file.endswith(".rgd")]

count = 0
# Process each .rgd file
for rgd_file_name in rgd_files:
    print(f"Processing file: {rgd_file_name}")

    with open(rgd_file_name, "r", encoding="latin-1") as rgd_file:
        rgd_content = rgd_file.readlines()

    # Update ReplayID with the current file name (without extension)
    replay_id = os.path.splitext(rgd_file_name)[0]
    replay_name = replay_id.split(".")[0]
    upgrade_actions = parse_rgd_file(rgd_content, replay_name)

    # Sample code for creating DataFrames
    upgrade_df = pd.DataFrame(upgrade_actions)

    if upgrade_df.empty:
        upgrade_df = pd.DataFrame(
            columns=[
                "PlayerID",
                "ActionType",
                "UpgradeType",
                "UpgradeLevel",
                "StartFrame",
                "EndFrame",
            ]
        )

    # Save DataFrames to CSV files
    upgrade_df.to_csv(f"{replay_name}_upgrades.csv", index=False)


    print(f"File {rgd_file_name} processed.\n")
    count+= 1
    # print(upgrade_df)
    # break

print("Total Files Processed: ", count)

Processing file: GG10.rep.rgd
File GG10.rep.rgd processed.

Processing file: GG10042.rep.rgd
File GG10042.rep.rgd processed.

Processing file: GG10043.rep.rgd
File GG10043.rep.rgd processed.

Processing file: GG10056.rep.rgd
File GG10056.rep.rgd processed.

Processing file: GG1010.rep.rgd
File GG1010.rep.rgd processed.

Processing file: GG10208.rep.rgd
File GG10208.rep.rgd processed.

Processing file: GG10209.rep.rgd
File GG10209.rep.rgd processed.

Processing file: GG1059.rep.rgd
File GG1059.rep.rgd processed.

Processing file: GG10646.rep.rgd
File GG10646.rep.rgd processed.

Processing file: GG1066.rep.rgd
File GG1066.rep.rgd processed.

Processing file: GG10951.rep.rgd
File GG10951.rep.rgd processed.

Processing file: GG11.rep.rgd
File GG11.rep.rgd processed.

Processing file: GG11048.rep.rgd
File GG11048.rep.rgd processed.

Processing file: GG11049.rep.rgd
File GG11049.rep.rgd processed.

Processing file: GG11050.rep.rgd
File GG11050.rep.rgd processed.

Processing file: GG11091.rep

# Research Conversion

In [50]:
import os
import pandas as pd


def parse_rgd_file(rgd_content, replay_id):
    research_actions = []
    for line in rgd_content:
        action_info = line.strip().split(",")
        # print(action_info)

        if (len(action_info)>2):
            columns = action_info
            # print(action_info)
            if columns[2] == "StartResearch":
                # print(action_info)
                # Extract relevant information for FinishResearch action
                start_frame = int(columns[0])
                player_id = int(columns[1])
                research_type = columns[3]
                # Append StartUpgrade data to the list
                research_actions.append({
                    "PlayerID": player_id + 1,
                    "ActionType": "Research",
                    "ResearchType": research_type,
                    "StartFrame": start_frame,
                    "EndFrame": -1
                })
                # Check if the ActionType is "FinishUpgrade"
            elif columns[2] == "FinishResearch":
                # Find the corresponding StartUpgrade action
                # print(upgrade_actions)
                for i in range(len(research_actions)):
                    if research_actions[i]["PlayerID"] == int(columns[1]) + 1 and research_actions[i]["ResearchType"] == columns[3]:
                        # Update the corresponding StartUpgrade action with FinishUpgrade data
                        research_actions[i]["EndFrame"] = int(columns[0])

    return research_actions


# Get a list of all .rgd files in the directory
rgd_files = [file for file in os.listdir() if file.endswith(".rgd")]

count = 0
# Process each .rgd file
for rgd_file_name in rgd_files:
    print(f"Processing file: {rgd_file_name}")

    with open(rgd_file_name, "r", encoding="latin-1") as rgd_file:
        rgd_content = rgd_file.readlines()

    # Update ReplayID with the current file name (without extension)
    replay_id = os.path.splitext(rgd_file_name)[0]
    replay_name = replay_id.split(".")[0]
    research_actions = parse_rgd_file(rgd_content, replay_name)

    # Sample code for creating DataFrames
    research_df = pd.DataFrame(research_actions)

    if research_df.empty:
        research_df = pd.DataFrame(
            columns=[
                "PlayerID",
                "ActionType",
                "ResearchType",
                "StartFrame",
                "EndFrame"
            ]
        )

    # Save DataFrames to CSV files
    research_df.to_csv(f"{replay_name}_researches.csv", index=False)


    print(f"File {rgd_file_name} processed.\n")
    count+= 1
    # print(upgrade_df)
    # break

print("Total Files Processed: ", count)

Processing file: GG10.rep.rgd
File GG10.rep.rgd processed.

Processing file: GG10042.rep.rgd
File GG10042.rep.rgd processed.

Processing file: GG10043.rep.rgd
File GG10043.rep.rgd processed.

Processing file: GG10056.rep.rgd
File GG10056.rep.rgd processed.

Processing file: GG1010.rep.rgd
File GG1010.rep.rgd processed.

Processing file: GG10208.rep.rgd
File GG10208.rep.rgd processed.

Processing file: GG10209.rep.rgd
File GG10209.rep.rgd processed.

Processing file: GG1059.rep.rgd
File GG1059.rep.rgd processed.

Processing file: GG10646.rep.rgd
File GG10646.rep.rgd processed.

Processing file: GG1066.rep.rgd
File GG1066.rep.rgd processed.

Processing file: GG10951.rep.rgd
File GG10951.rep.rgd processed.

Processing file: GG11.rep.rgd
File GG11.rep.rgd processed.

Processing file: GG11048.rep.rgd
File GG11048.rep.rgd processed.

Processing file: GG11049.rep.rgd
File GG11049.rep.rgd processed.

Processing file: GG11050.rep.rgd
File GG11050.rep.rgd processed.

Processing file: GG11091.rep